In [1]:
import pandas as pd
import os
print(os.getcwd())
from jenga.corruptions import DuckDBCorruptionBrokenCharacters

/home/cmol/Documents/University/ComputerScience2023-2024/DataPreperation/DataPreperationProject


In [3]:
df = pd.read_json("./data/small.json.gz", compression="gzip", lines=True)
df

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote
0,5,True,"11 19, 2014",A1QVBUH9E1V6I8,4639725183,Jamshed Mathur,No adverse comment.,Five Stars,1416355200,NaN
1,5,True,"10 13, 2016",A3GEOILWLK86XM,4639725183,itsjustme,Gift for college student.,Great product.,1476316800,NaN
2,5,True,"11 21, 2015",A32RD6L701BIGP,4639725183,Krystal Clifton,"If you like strong tea, this is for you. It mi...",Strong,1448064000,NaN
3,5,True,"08 12, 2015",A2UY1O1FBGKIE6,4639725183,U. Kane,Love the tea. The flavor is way better than th...,Great tea,1439337600,NaN
4,5,True,"05 28, 2015",A3QHVBQYDV7Z6U,4639725183,The Nana,I have searched everywhere until I browsed Ama...,This is the tea I remembered!,1432771200,NaN
...,...,...,...,...,...,...,...,...,...,...
112,5,True,"10 28, 2014",A3HKBY3DH96VVT,9742356831,Icydawg,as described and shipped on time tastey,Five Stars,1414454400,NaN
113,4,True,"10 1, 2014",A1FIOBU4XSMO56,9742356831,WRS,Good stuff. Would order again.,Very good.,1412121600,NaN
114,5,True,"09 22, 2014",A4EWRIA8CCRMQ,9742356831,Ann,yum,Five Stars,1411344000,NaN
115,4,True,"09 13, 2014",A9DIYBDBJDOJ,9742356831,Blue Dragonfly,I have Mae Ploy Yellow Curry and Panang Red Cu...,Very hot!,1410566400,13.0


In [4]:
duckdb_corruption = DuckDBCorruptionBrokenCharacters("reviewText", 0.2)
duckdb_corruption.transform(df)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote
0,5,True,"01 11, 2018",A3VG446SHJH37O,9742356831,PLee,Absolutely LOVE this!!! It's spicy but sooooo ...,LOVE!,1515628800,NaN
1,5,True,"07 13, 2016",A3LXB49N4XLHUC,9742356831,Cosala Salaci,Best green curry paste. A little goes a long w...,Delicious,1468368000,NaN
2,5,True,"11 22, 2014",ARW3VESFEOKOA,9742356831,Cherie,Best there is for green curry.,Five Stars,1416614400,NaN
3,5,True,"09 5, 2016",A36AKNBXEYHNNS,9742356831,TBT,"By fár thé bést pástéd I'vé úséd, évén béttér ...",By far the best pasted I've used,1473033600,NaN
4,5,True,"08 28, 2016",A32TZ4HWVHBYX7,4639725043,cowgirl,Bést téá fớr my singlé cúp cớfféé mákér. I púr...,Best tea for my single cup coffee maker,1472342400,NaN
...,...,...,...,...,...,...,...,...,...,...
112,5,True,"08 30, 2017",A16Q398S571Y58,9742356831,SSS,tastes authentic,Five Stars,1504051200,NaN
113,5,True,"05 5, 2014",A1T65DGV69QI8L,4639725043,flying squirrel girl,"this tea is the bomb, for the real die-hard in...",the bomb,1399248000,5.0
114,5,True,"09 22, 2014",A4EWRIA8CCRMQ,9742356831,Ann,yum,Five Stars,1411344000,NaN
115,5,True,"06 20, 2017",A17FA8KTT1W2EL,9742356831,MafiaMom,"Áctúálly véry déliciớús, éspéciálly with méát ...",Delicious!,1497916800,NaN
